In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
sys.path.append('../src')

from lm_polygraph.utils.model import WhiteboxModel
from lm_polygraph.estimators import *
from lm_polygraph.stat_calculators import *
from lm_polygraph.utils.openai_chat import OpenAIChat
from lm_polygraph.utils.deberta import Deberta

/home/ruixing/.conda/envs/poly/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ruixing/.conda/envs/poly/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/ruixing/.conda/envs/poly/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/ruixing/.conda/envs/poly/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_

In [3]:
bloomz="bigscience/bloomz-560m"
yi6bchat="01-ai/Yi-6B-Chat"
yi6b="01-ai/Yi-6B"
t5base='google-t5/t5-base'
model = WhiteboxModel.from_pretrained(yi6b)

/home/ruixing/.conda/envs/poly/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.71s/it]


In [16]:
texts = ["请介绍一下曹操。"]
stat = {}

# MBZ Budget
os.environ["OPENAI_KEY"] = "YOUR_API_KEY"

for calculator in [
    GreedyProbsCalculator(),
    EntropyCalculator(),
    GreedyLMProbsCalculator(),
    # ClaimsExtractorZH(OpenAIChat("gpt-4o")),
]:
    stat.update(calculator(stat, texts, model))

claim_extractor=ClaimsExtractor(OpenAIChat("gpt-4o"))
stat.update(claim_extractor(stat, texts, model,language="zh"))

In [17]:
stat.keys()

dict_keys(['input_texts', 'input_tokens', 'greedy_log_probs', 'greedy_tokens', 'greedy_tokens_alternatives', 'greedy_texts', 'greedy_log_likelihoods', 'embeddings_decoder', 'entropy', 'greedy_lm_log_probs', 'greedy_lm_log_likelihoods', 'claims', 'claim_texts_concatenated', 'claim_input_texts_concatenated'])

In [18]:
stat['claims']

[[Claim(claim_text='曹操（155年－220年3月15日）。', sentence='曹操（155年－220年3月15日），字孟德，小字阿瞒，沛国谯县（今安徽亳州）人', aligned_tokens=[1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]),
  Claim(claim_text='曹操字孟德。', sentence='曹操（155年－220年3月15日），字孟德，小字阿瞒，沛国谯县（今安徽亳州）人', aligned_tokens=[1, 19, 20, 21]),
  Claim(claim_text='曹操小字阿瞒。', sentence='曹操（155年－220年3月15日），字孟德，小字阿瞒，沛国谯县（今安徽亳州）人', aligned_tokens=[1, 23, 24, 25, 26]),
  Claim(claim_text='曹操是沛国谯县（今安徽亳州）人。', sentence='曹操（155年－220年3月15日），字孟德，小字阿瞒，沛国谯县（今安徽亳州）人', aligned_tokens=[1, 28, 29]),
  Claim(claim_text='东汉末年杰出的政治家。', sentence='东汉末年杰出的政治家、军事家、文学家、书法家，三国中曹魏政权的奠基人', aligned_tokens=[44, 45, 46, 47, 48, 49, 50, 51]),
  Claim(claim_text='东汉末年杰出的军事家。', sentence='东汉末年杰出的政治家、军事家、文学家、书法家，三国中曹魏政权的奠基人', aligned_tokens=[44, 45, 46, 47, 48, 49, 53, 54]),
  Claim(claim_text='东汉末年杰出的文学家。', sentence='东汉末年杰出的政治家、军事家、文学家、书法家，三国中曹魏政权的奠基人', aligned_tokens=[44, 45, 46, 47, 48, 49, 56, 57]),
  Claim(claim_text='东汉末年杰出的书法家。', sentence='东汉末年杰出的政治家、军事家、文学家、书法家，三国中曹魏政权的奠基人', align

In [19]:
print("Output:", stat["greedy_texts"][0])

Output: 
曹操（155年－220年3月15日），字孟德，小字阿瞒，沛国谯县（今安徽亳州）人。东汉末年杰出的政治家、军事家、文学家、书法家，三国中曹魏政权的奠基人。
曹操在东汉末年杰出的政治家、军事家、文学家、书法家，三国中曹魏政权的奠


In [20]:
print("Output:", stat["greedy_texts"][0])
print()
for claim in stat["claims"][0]:
    print("claim:", claim.claim_text)
    print("aligned tokens:", claim.aligned_tokens)
    print()

Output: 
曹操（155年－220年3月15日），字孟德，小字阿瞒，沛国谯县（今安徽亳州）人。东汉末年杰出的政治家、军事家、文学家、书法家，三国中曹魏政权的奠基人。
曹操在东汉末年杰出的政治家、军事家、文学家、书法家，三国中曹魏政权的奠

claim: 曹操（155年－220年3月15日）。
aligned tokens: [1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

claim: 曹操字孟德。
aligned tokens: [1, 19, 20, 21]

claim: 曹操小字阿瞒。
aligned tokens: [1, 23, 24, 25, 26]

claim: 曹操是沛国谯县（今安徽亳州）人。
aligned tokens: [1, 28, 29]

claim: 东汉末年杰出的政治家。
aligned tokens: [44, 45, 46, 47, 48, 49, 50, 51]

claim: 东汉末年杰出的军事家。
aligned tokens: [44, 45, 46, 47, 48, 49, 53, 54]

claim: 东汉末年杰出的文学家。
aligned tokens: [44, 45, 46, 47, 48, 49, 56, 57]

claim: 东汉末年杰出的书法家。
aligned tokens: [44, 45, 46, 47, 48, 49, 59, 60]

claim: 三国中曹魏政权的奠基人。
aligned tokens: [62, 63, 64, 65, 66, 67, 68, 69, 70]



In [23]:
# Maximum Claim Probability
max_prob = MaximumClaimProbability()
max_prob(stat)  # Uncertainty for each claim, the higher, the less certain

[[2.059152,
  1.1362455,
  2.9324822,
  1.4121381,
  0.7077023,
  0.69397306,
  0.7427101,
  0.69853115,
  0.5728888]]

In [24]:
# Perlexity
perlexity_claim = PerplexityClaim()
perlexity_claim (stat)  

[[0.1372768,
  0.28406137,
  0.5864965,
  0.4707127,
  0.088462785,
  0.08674663,
  0.092838764,
  0.087316394,
  0.06365431]]

In [25]:
# Maximum Token Entropy
max_token_ent = MaxTokenEntropyClaim()
max_token_ent(stat)  

[[7.67348e-05,
  7.67348e-05,
  7.67348e-05,
  7.67348e-05,
  2.765898e-05,
  2.765898e-05,
  2.765898e-05,
  2.765898e-05,
  1.8441091e-05]]

In [26]:
# Pointwise Mutual Information
pmi = PointwiseMutualInformationClaim()
pmi (stat)  

[[-72.85801413972513,
  -30.04430177868926,
  -47.79385693371296,
  -26.227787380106747,
  -107.58818900730694,
  -107.45839975989657,
  -108.33176562050357,
  -106.02369641518453,
  -128.93925437084226]]

In [ ]:
# PTrue
ptrue_claim = PTrueClaim()
ptrue_claim (stat)  

In [27]:
# Claim Conditional Probability
for calculator in [
    GreedyAlternativesNLICalculator(Deberta(deberta_path="MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7"))
]:
    stat.update(calculator(stat, texts, model))

ccp = ClaimConditionedProbabilityClaim()
ccp (stat)

Some weights of the model checkpoint at microsoft/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[[-0.7113813961415233,
  -0.8446114377545357,
  -0.2803199309110498,
  -0.7791504012475273,
  -0.5517237721010816,
  -0.557831080274213,
  -0.5316904995544154,
  -0.5553218365419984,
  -0.7292863406843685]]

In [28]:
for calculator in [
    GreedyAlternativesFactPrefNLICalculator(Deberta(deberta_path="MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7"))
]:
    stat.update(calculator(stat, texts, model))

ccp_no_cxt=ClaimConditionedProbabilityClaim(nli_context="fact_pref")
ccp_no_cxt (stat)

Some weights of the model checkpoint at microsoft/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[[-0.7235015435870603,
  -0.8481761220949592,
  -0.39506225276896145,
  -0.7651305626056635,
  -0.5732555949369494,
  -0.5790878572436547,
  -0.5517374369245559,
  -0.5765155444379375,
  -0.6967716564740467]]

In [21]:
from lm_polygraph.generation_metrics.openai_fact_check import OpenAIFactCheck
chinese_checker = OpenAIFactCheck('gpt-4o')
chatgpt_response = chinese_checker(stat, None, None, language="zh")

In [22]:
chatgpt_response

[[0, 0, 0, 0, 0, 0, 0, nan, 0]]